## Helpers

In [37]:
import json

def pretty_print(arg):
    print(json.dumps(arg, indent=4, sort_keys=True))
    
pretty_print({
    'one':1,
    'two': [1,2],
    'three':{'1': 1,'2':2,'3':3}
})


{
    "one": 1,
    "three": {
        "1": 1,
        "2": 2,
        "3": 3
    },
    "two": [
        1,
        2
    ]
}


In [38]:
import os

def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0

In [39]:
def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)


# Lets check the file size of MS Paint exe
# or you can use any file path
file_path = 'job_list.ipynb'
print (file_size(file_path))


30.8 KB


## Client Folder Path

In [40]:
# path = "/users/glweems/Desktop/clients"
path = "/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients"


In [224]:
import os
from glob import glob
import pdfrw
from pdfrw import PdfReader

def builder(path):
    paths = os.listdir(path)
    obj = {}
    for item in paths:
        i = 0
        jobs = {}
        dir = os.path.join(path, item)
        _jobs = glob(dir+'*/**/*/PRINT', recursive=True)
        default = glob(dir+'/PRINT/*.pdf', recursive=True)
        if len(default) > 0:
             for file in default: 
                media_box = PdfReader(file).pages[0].MediaBox
                file_obj = {
                    'full_path': file,
                    'file_name': file.split('/').pop(),
                    'width': round(float(media_box[2])/72, 2),
                    'height': round(float(media_box[3])/72, 2),
                    'size': file_size(file)
                }

                qty = file_obj['file_name'].replace('.pdf','').split('qty_')
                try:
                    qty = int(qty[1])
                except IndexError:
                    qty = 1
                    
                file_obj['qty'] = qty
                    
                # print(int(qty)
                files[file_obj['file_name']] = file_obj
                jobs['.'] = {
                    'full_path': job_full_path,
                    'files': files
                }
        # print()
        # try: 
        #     print(jobs[0])
        # except: 
        
        # print(_jobs)
        for job in _jobs:
            job_full_path = job
            job = job.replace(dir+'/', '').replace('/PRINT', '')
            files = {}

            for file in glob(job_full_path+'/*.pdf'):
                media_box = PdfReader(file).pages[0].MediaBox
                file_obj = {
                    'full_path': file,
                    'file_name': file.split('/').pop(),
                    'width': round(float(media_box[2])/72, 2),
                    'height': round(float(media_box[3])/72, 2),
                    'size': file_size(file)
                }

                qty = file_obj['file_name'].replace('.pdf','').split('qty_')
                try:
                    qty = int(qty[1])
                except IndexError:
                    qty = 1
                    
                file_obj['qty'] = qty
                    
                # print(int(qty)
                files[file_obj['file_name']] = file_obj

            jobs[job] = {
                'full_path': job_full_path,
                'files': files
            }

        obj[item] = {
            'full_path': dir,
            'jobs': jobs

        }
    return obj


clients = builder(path)


In [233]:
data = []


for client in clients:
    client_obj = clients[client]
    
    for job in client_obj['jobs']:
        job_obj = client_obj['jobs'][job]
        
        for file in job_obj['files']:
            file_obj = job_obj['files'][file]
            
            
            rowobj = {
                'client': client,
                'project': ('', job)[job != '.'],
                'file_name': file,
                'file_size': file_obj['size'],
                'file_path': file_obj['full_path'],
                'height': file_obj['height'],
                'width': file_obj['width'],
                # 'sqft': 
                'qty': file_obj['qty'],
                'material': '',
            }
            data.append(rowobj)

In [232]:
import pandas as pd

df = pd.DataFrame(data)
df.to_csv('myfile.csv')

# DB

In [151]:
database = r"buffalographics.db"


In [152]:
import sqlite3
from sqlite3 import Error


def create_connection(database):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(database)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection(database)


2.6.0


In [159]:
sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    priority integer,
                                    status_id integer NOT NULL,
                                    project_id integer NOT NULL,
                                    begin_date text NOT NULL,
                                    end_date text NOT NULL,
                                    FOREIGN KEY (project_id) REFERENCES clients (id)
                                );"""

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


In [154]:
def create_client(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO clients(name)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid


In [155]:
def create_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

    return cur.lastrowid


In [161]:
import sqlite3
from sqlite3 import Error


sql_create_clients_table = """ CREATE TABLE IF NOT EXISTS clients (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL
                                    ); """

def main():

    # create a database connection
    conn = create_connection(database)
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_clients_table)

        # create tasks table
        create_table(conn, sql_create_tasks_table)
    else:
        print("Error! cannot create the database connection.")
    with conn:
        # create a new project
        project = ('Cool App with SQLite & Python')
        project_id = create_project(conn, project)

        # tasks
        task_1 = ('Analyze the requirements of the app', 1,
                  1, project_id, '2015-01-01', '2015-01-02')
        task_2 = ('Confirm with user about the top requirements',
                  1, 1, project_id, '2015-01-03', '2015-01-05')

        # create tasks
        create_task(conn, task_1)
        create_task(conn, task_2)


if __name__ == '__main__':
    main()


2.6.0
Error! cannot create the database connection.


AttributeError: __enter__